In [225]:
import pandas as pd

####user
user = pd.read_csv('user.csv')
#print(user) 

####transaction
transaction = pd.read_csv('transaction.csv')
#transaction = df[df['user_id'].isin([1, 5])]

####join transaction with user
transaction_2 = pd.merge(transaction, user, on='user_id', how='inner')
print(transaction_2)


             date   hour  user_id      province
0      2021-09-16  18:00        1     Sukhothai
1      2021-09-17  06:00        1        Phayao
2      2021-09-16  23:00        1         Phrae
3      2021-09-16  11:00        1         Phrae
4      2021-11-20  04:00        1        Phayao
...           ...    ...      ...           ...
41690  2021-12-30  18:00     1000  Samut Sakhon
41691  2021-12-30  21:00     1000    Ratchaburi
41692  2021-12-31  09:00     1000       Bangkok
41693  2021-12-31  14:00     1000  Samut Prakan
41694  2021-12-31  23:00     1000       Bangkok

[41695 rows x 4 columns]


In [227]:
####flag data if province are home
transaction_2['Type'] = 'Null' 
transaction_2.loc[transaction_2['province'] == transaction_2['hometown'], 'Type'] = 'Home'
transaction_2.loc[transaction_2['province'] != transaction_2['hometown'], 'Type'] = 'Travel'
transaction_2.drop(columns=['hometown'], inplace=True) 
print(transaction_2)

             date   hour  user_id      province    Type
0      2021-09-16  18:00        1     Sukhothai  Travel
1      2021-09-17  06:00        1        Phayao  Travel
2      2021-09-16  23:00        1         Phrae    Home
3      2021-09-16  11:00        1         Phrae    Home
4      2021-11-20  04:00        1        Phayao  Travel
...           ...    ...      ...           ...     ...
41690  2021-12-30  18:00     1000  Samut Sakhon  Travel
41691  2021-12-30  21:00     1000    Ratchaburi  Travel
41692  2021-12-31  09:00     1000       Bangkok    Home
41693  2021-12-31  14:00     1000  Samut Prakan  Travel
41694  2021-12-31  23:00     1000       Bangkok    Home

[41695 rows x 5 columns]


In [228]:
####loop genarate Trip_id
transaction_3 = transaction_2.sort_values(by=['user_id','date','hour'], ascending=[True,True,True])
transaction_4 = transaction_3.reset_index(drop=True)

index = 0
Trip_id = 1
check_user_id = '0'
while index < len(transaction_3):  # Loop runs as long as count is less than 5

    check = transaction_4.iloc[index]
    
    if check['Type'] == 'Home' or check['user_id'] != check_user_id:
        Trip_id = Trip_id + 1
        check_user_id=check['user_id']
        

    transaction_4.loc[index, 'Trip_id'] = str(Trip_id) 
    index += 1   
print(transaction_4)

             date   hour  user_id      province    Type Trip_id
0      2021-09-16  11:00        1         Phrae    Home       2
1      2021-09-16  18:00        1     Sukhothai  Travel       2
2      2021-09-16  23:00        1         Phrae    Home       3
3      2021-09-17  06:00        1        Phayao  Travel       3
4      2021-11-19  13:00        1         Phrae    Home       4
...           ...    ...      ...           ...     ...     ...
41690  2021-12-30  21:00     1000    Ratchaburi  Travel   17075
41691  2021-12-30  23:00     1000   Phetchaburi  Travel   17075
41692  2021-12-31  09:00     1000       Bangkok    Home   17076
41693  2021-12-31  14:00     1000  Samut Prakan  Travel   17076
41694  2021-12-31  23:00     1000       Bangkok    Home   17077

[41695 rows x 6 columns]


In [229]:
####Drop record if Type = 'Home'
transaction_5 = transaction_4.loc[transaction_4['Type'] != 'Home']
transaction_5

,date,hour,user_id,province,Type,Trip_id
1,2021-09-16,18:00,1,Sukhothai,Travel,2
3,2021-09-17,06:00,1,Phayao,Travel,3
5,2021-11-19,17:00,1,Sukhothai,Travel,4
6,2021-11-20,02:00,1,Lampang,Travel,4
7,2021-11-20,04:00,1,Phayao,Travel,4
...,...,...,...,...,...,...
41687,2021-12-29,21:00,1000,Pathum Thani,Travel,17074
41689,2021-12-30,18:00,1000,Samut Sakhon,Travel,17075
41690,2021-12-30,21:00,1000,Ratchaburi,Travel,17075
41691,2021-12-30,23:00,1000,Phetchaburi,Travel,17075


In [230]:
####Find start_date and end_date
transaction_5['start_date'] = transaction_5.groupby('Trip_id')['date'].transform('min')
transaction_5['end_date'] = transaction_5.groupby('Trip_id')['date'].transform('max')
transaction_5

C:\Users\var_m\AppData\Local\Temp\ipykernel_15488\1214036842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_5['start_date'] = transaction_5.groupby('Trip_id')['date'].transform('min')
C:\Users\var_m\AppData\Local\Temp\ipykernel_15488\1214036842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_5['end_date'] = transaction_5.groupby('Trip_id')['date'].transform('max')


,date,hour,user_id,province,Type,Trip_id,start_date,end_date
1,2021-09-16,18:00,1,Sukhothai,Travel,2,2021-09-16,2021-09-16
3,2021-09-17,06:00,1,Phayao,Travel,3,2021-09-17,2021-09-17
5,2021-11-19,17:00,1,Sukhothai,Travel,4,2021-11-19,2021-11-20
6,2021-11-20,02:00,1,Lampang,Travel,4,2021-11-19,2021-11-20
7,2021-11-20,04:00,1,Phayao,Travel,4,2021-11-19,2021-11-20
...,...,...,...,...,...,...,...,...
41687,2021-12-29,21:00,1000,Pathum Thani,Travel,17074,2021-12-29,2021-12-29
41689,2021-12-30,18:00,1000,Samut Sakhon,Travel,17075,2021-12-30,2021-12-30
41690,2021-12-30,21:00,1000,Ratchaburi,Travel,17075,2021-12-30,2021-12-30
41691,2021-12-30,23:00,1000,Phetchaburi,Travel,17075,2021-12-30,2021-12-30


In [231]:
####list file same Trip_id
transaction_6 = ( 
            transaction_5
            .groupby(['Trip_id','user_id','start_date','end_date'])
            .agg({'province': ', '.join})
            .sort_values(by=['user_id','start_date'], ascending=[True,True])
            .reset_index()
)
transaction_6['province'] = transaction_6['province'].apply(lambda x: ', '.join(sorted(set(x.split(', ')))))
#transaction_6['Trip_id'] = transaction_6.reset_index().index + 1
transaction_6

,Trip_id,user_id,start_date,end_date,province
0,2,1,2021-09-16,2021-09-16,Sukhothai
1,3,1,2021-09-17,2021-09-17,Phayao
2,4,1,2021-11-19,2021-11-20,"Chiang Rai, Lampang, Phayao, Sukhothai"
3,5,1,2021-12-20,2021-12-20,Lampang
4,6,1,2021-12-25,2021-12-25,"Bangkok, Nonthaburi"
...,...,...,...,...,...
10901,17071,1000,2021-12-23,2021-12-23,Samut Prakan
10902,17073,1000,2021-12-24,2021-12-24,Pathum Thani
10903,17074,1000,2021-12-29,2021-12-29,Pathum Thani
10904,17075,1000,2021-12-30,2021-12-30,"Phetchaburi, Ratchaburi, Samut Sakhon"


In [232]:
####Re-generate Trip_id
transaction_6['Trip_id'] = (transaction_6.reset_index().index + 1).astype(str).str.zfill(4)
transaction_6['user_id'] = transaction_6['user_id'].astype(str).str.zfill(4)
transaction_6

,Trip_id,user_id,start_date,end_date,province
0,0001,0001,2021-09-16,2021-09-16,Sukhothai
1,0002,0001,2021-09-17,2021-09-17,Phayao
2,0003,0001,2021-11-19,2021-11-20,"Chiang Rai, Lampang, Phayao, Sukhothai"
3,0004,0001,2021-12-20,2021-12-20,Lampang
4,0005,0001,2021-12-25,2021-12-25,"Bangkok, Nonthaburi"
...,...,...,...,...,...
10901,10902,1000,2021-12-23,2021-12-23,Samut Prakan
10902,10903,1000,2021-12-24,2021-12-24,Pathum Thani
10903,10904,1000,2021-12-29,2021-12-29,Pathum Thani
10904,10905,1000,2021-12-30,2021-12-30,"Phetchaburi, Ratchaburi, Samut Sakhon"


In [233]:
transaction_6.to_csv('summary_transaction.csv', index=False)